In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import PreClf, MultiPreClf
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt 

In [ ]:
# dummy data for perceptron training
np.random.seed(42)

X = np.random.randn(100, 2) # 100 points with 2 features
y = np.where(X[:, 0] + X[:, 1] > 1, 1, 0) # Class 1 if x1 + x2 > 1, else class 0

data = pd.DataFrame(X, columns=['x1', 'x2'])
data['y'] = y

# MNIST 

X_, y_ = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False, parser="pandas")


# Preprocessing normalisation 
X_ = X_ / 255



In [70]:
prctrn = PreClf.SingleLayerPreceptronClassifier(alpha=0.1, n_iterations=10)

In [71]:
prctrn.fit(X, y)

In [72]:
prctrn.cost_history

[12.0, 8.0, 5.0, 2.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0]

In [19]:
X_[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [11]:
mlprtn = MultiPreClf.MultilayerPerceptron(alpha=0.01, n_iterations=50, input_layer_size=784, hidden_layers_size=[64, 32], n_outputs=10)

In [44]:
mlprtn.forward_propagation(X_[0])

Layer: 0, x: (784,), w: (784, 64), b: (1, 64)
Linear part output shape: (1, 64)
Activation part output shape: (1, 64)
Layer: 1, x: (1, 64), w: (64, 32), b: (1, 32)
Linear part output shape: (1, 32)
Activation part output shape: (1, 32)
Last linear part output shape: (1, 10)
Last activation part output shape: (1, 10)


m:\ML\ML_regs\Preceptron\MultiPreClf.py:86: RuntimeWarning: overflow encountered in exp


: 

In [37]:

X_[0].reshape(1, -1).shape

(1, 784)

In [42]:
mlprtn.weights[0].shape

(784, 64)

In [28]:
# X_ shape is (784, ) and weights[0] shape is (784, 64)
np.dot(X_[0].ravel(), mlprtn.weights[0]).shape

(64,)

In [31]:
(X_[0] @ mlprtn.weights[0]).shape

(64,)

In [32]:
X_[0].dot(mlprtn.weights[0]).shape

(64,)